In [1]:
import pandas as pd
import folium
import numpy as np
import time
from haversine import haversine_vector
from itertools import combinations
from easydict import EasyDict
from apis import *

In [2]:
df = pd.read_pickle("~/bdbd_datateam_research/data/recommend_table_v1.0.pkl")

In [3]:
map = folium.Map(location=[37.541, 126.986], zoom_start=12)

In [4]:
pointA = (127.05885852710445, 37.51224523081787)  # 코엑스
pointB = (126.9648349524943, 37.530706461090965)  # 용산역
pointC = (127.04720825721974, 37.54715522929517)  # 뚝섬역

## GPS 좌표 직선거리 이용한 반경이내 아파트 추천 방식
각 지점마다 서치할 반경은 다르게 줄 수 있음. 이걸로 가중치 반영가능할 듯함.

In [5]:
# 직선거리로 반경 안에 있는 아파트 소트하는 경우
results = []
for name, x, y in zip(df['단지구역명'], df['단지X좌표'], df['단지Y좌표']):
    if check_if_intersect((x, y), (pointA, 6), (pointB, 6), (pointC, 6)):
        results.append((name, x, y))

len(results)

189

In [6]:
for point in [pointA, pointB, pointC]:
    folium.Circle(
        location = (point[1], point[0]),
        tooltip = "PointA",
        radius = 6000,
        fill=True,
        color='blue'
    ).add_to(map)

    folium.Marker(
        location=(point[1], point[0]),
        popup=name,
        icon=folium.Icon(color='blue', icon='star')
    ).add_to(map)

In [7]:
for name, x, y in results:
    folium.Marker(
        location=(y, x),
        popup=name,
        icon=folium.Icon(color='red', icon='home')
    ).add_to(map)

In [8]:
map

## 지하철역 기준으로 아파트 검색가능하도록 클러스터링 해보기
지하철역 to 지하철역 이동 시간 정보 테이블 만들어서 해당 지하철역에서 일정 거리 안에 검색가능한 형태로

In [9]:
subway = pd.read_csv('~/bdbd_datateam_research/data/kms_tables/지하철코드정보.csv', sep="|")

In [10]:
subway.columns

Index(['Unnamed: 0', '지하철코드', '호선명', '역명', '사용여부', '등록일자', '수정일자', 'X좌표',
       'Y좌표', '환승가능여부', '법정동코드', '지역명', '간략호선명', '호선색상코드', '처리요청구분자', '요청일자',
       '처리일자'],
      dtype='object')

In [11]:
subway[subway['지하철코드'].str.contains(r'[a-zA-Z]')]['호선명'].unique()

array(['서울-1호선', '서울-5호선', '수인분당선', '경의중앙선', '경춘선', '신분당선', '의정부경전철',
       '에버라인', '경강선', '우이신설경전철', '서해선', '김포골드라인', '부산-김해경전철', '동해선',
       '인천-1호선', '인천-2호선', '공항철도', '인천공항자기부상'], dtype=object)

In [12]:
# 서울 지하철 역만해도 벌써 753개 인데, 753 x 753 하면 5만6천개정도 되서 api 결재 안하면 할 수가 없네...
subway[subway['지역명'] == '수도권']

,Unnamed: 0,지하철코드,호선명,역명,사용여부,등록일자,수정일자,X좌표,Y좌표,환승가능여부,법정동코드,지역명,간략호선명,호선색상코드,처리요청구분자,요청일자,처리일자
0,0,1101000,서울-1호선,소요산,Y,2010-05-19 11:21:13,2010-05-19 11:21:13,127.061032,37.948617,N,4125010900,수도권,1호선,#0d3692,Y,NaN,NaN
1,1,1101010,서울-1호선,동두천,Y,2010-05-19 11:21:13,2010-05-19 11:21:13,127.054845,37.927573,N,4125010700,수도권,1호선,#0d3692,Y,NaN,NaN
2,2,1101020,서울-1호선,보산,Y,2010-05-19 11:21:13,2010-05-19 11:21:13,127.057158,37.914277,N,4125010600,수도권,1호선,#0d3692,Y,NaN,NaN
3,3,1101030,서울-1호선,동두천중앙,Y,2010-05-19 11:21:13,2010-05-19 11:21:13,127.056433,37.901682,N,4125010300,수도권,1호선,#0d3692,Y,NaN,NaN
4,4,1101040,서울-1호선,지행,Y,2010-05-19 11:21:13,2010-05-19 11:21:13,127.055757,37.892369,N,4125010200,수도권,1호선,#0d3692,Y,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
988,988,4BI1110,인천공항자기부상,장기주차장,Y,2017-03-31 00:00:00,2017-03-31 00:00:00,126.455582,37.443909,N,2811014700,수도권,인천공항자기부상,#ffcd12,Y,NaN,NaN
989,989,4BI1120,인천공항자기부상,합동청사,Y,2017-03-31 00:00:00,2017-03-31 00:00:00,126.458641,37.440529,N,2811014700,수도권,인천공항자기부상,#ffcd12,Y,NaN,NaN
990,990,4BI1130,인천공항자기부상,국제업무단지,Y,2017-03-31 00:00:00,2017-03-31 00:00:00,126.459831,37.437208,N,2811014700,수도권,인천공항자기부상,#ffcd12,Y,NaN,NaN
991,991,4BI1140,인천공항자기부상,워터파크,Y,2017-03-31 00:00:00,2017-03-31 00:00:00,126.434183,37.429198,N,2811014700,수도권,인천공항자기부상,#ffcd12,Y,NaN,NaN


In [13]:
new_sub = subway[['지하철코드', '호선명', '간략호선명', '호선색상코드', '역명', 'X좌표', 'Y좌표', '지역명']]
new_sub.columns = ['code', 'line', 'line_short', 'line_color', 'station_name', 'X', 'Y', 'region']
#new_sub.set_index('code', inplace=True)

In [14]:
#new_sub.to_pickle('subway_info.pkl')

In [15]:
sf = pd.read_pickle("~/bdbd_datateam_research/data/station_to_station_Gwangju_v1.2.pkl")

In [16]:
from accessibility import *

In [17]:
pointA = (126.93222940515193, 35.13275605066436)
pointB = (126.80040831732504, 35.1426913510177)
recommender = AptRecommenderByAccess(Aattr, AtoS, StoS, Sinfo, k_for_clusters=3)
result = recommender(pointA, pointB)

In [18]:
clusters = recommender.cluster_subs

In [19]:
clusters

,line,line_short,line_color,station_name,X,Y,region
code,,,,,,,
5101100,광주-1호선,1호선,#009088,돌고개,126.895578,35.151872,광주
5101110,광주-1호선,1호선,#009088,농성,126.883903,35.153300,광주
5101090,광주-1호선,1호선,#009088,양동시장,126.901319,35.154335,광주


In [25]:
map = folium.Map(location=[35.161047020112825, 126.86084212717452], zoom_start=13)

In [26]:
folium.Marker(
    location=(pointA[1], pointA[0]),
    tooltip='pointA',
    icon=folium.Icon(color='blue', icon='start')
).add_to(map)
folium.Marker(
    location=(pointB[1], pointB[0]),
    tooltip='pointB',
    icon=folium.Icon(color='blue', icon='star')
).add_to(map)

In [27]:
for cluster in clusters.index:
    apts = result[cluster]
    center_coords = (clusters.loc[cluster].Y, clusters.loc[cluster].X)
    coords = list(zip(apts['단지Y좌표'], apts['단지X좌표'], apts['단지구역명']))
    
    folium.Circle(
        location = center_coords,
        tooltip = "Station: r'{cluster}",
        radius = 600,
        fill=True
        ).add_to(map)
    for coord in coords:
        folium.Marker(
            location=coord[0:2],
            tooltip=coord[2],
            icon=folium.Icon(color='red', icon='home')
        ).add_to(map)

In [28]:

map

In [29]:
recommender.score_for_clusters

지하철코드
5101020    0.152588
5101030    0.199298
5101040    0.250000
5101050    0.176400
5101060    0.115600
5101070    0.048400
5101080    0.025600
5101120    0.010000
5101130    0.032400
5101140    0.057600
5101150    0.102400
5101160    0.129600
5101170    0.176400
5101180    0.250000
5101190    0.185791
5101200    0.143480
5101210    0.084505
dtype: float64

In [46]:
geo = [[127.04932105238444,37.503578820002986],[127.03400633104191,37.50209706130585],[127.03364399154539,37.502660024197525],[127.03281124730785,37.5036916999686],[127.03174237578986,37.504575882002506],[127.03047785452573,37.50527908058585],[127.02906557586661,37.505774660186056],[127.02755903255422,37.506043848951755],[127.0260152904837,37.506076450242354],[127.02449282564788,37.50587122914515],[127.02304930747367,37.5054359592869],[127.02173941282274,37.504787128160174],[127.02061275374984,37.50394931216701],[127.01971199773877,37.50295424513314],[127.01907125174655,37.50183961568524],[127.01871477127239,37.50064763916382],[127.01865604323426,37.499423458276624],[127.01889727716903,37.49821343315767],[127.02184113855108,37.48922762394294],[127.02245457630151,37.487947281624145],[127.02533988945413,37.48346545354202],[127.02627616287465,37.482331767353436],[127.02749652310368,37.481383389461996],[127.02894439489185,37.480664278713306],[127.03055266374963,37.48020776595467],[127.05321991354072,37.47588611970316],[127.05515380697224,37.47571207625453],[127.06080623622488,37.475757071645525],[127.07676691728192,37.47588236818898],[127.08526946819376,37.46757862140133],[127.08776979463369,37.46368850851576],[127.0886793523177,37.462579383080616],[127.10011666533582,37.45140397660267],[127.10120977135234,37.450527341873354],[127.10249719776216,37.44983674201861],[127.10392952102266,37.4493586862698],[127.10545175929698,37.44911152475639],[127.10700548183576,37.449104744615205],[127.10853105028013,37.44933860609501],[127.1282555985805,37.45399023073531],[127.12960607991576,37.4544190311244],[127.13083520888374,37.45503640960926],[127.13190164159016,37.45582160133466],[127.132769503107,37.45674819649553],[127.14669253416474,37.47487140141064],[127.14705275048622,37.47539432441513],[127.15537973923387,37.48896982345702],[127.15598526793943,37.49033873103978],[127.15617978634353,37.49178127511405],[127.15612926489287,37.49628627836077],[127.1559638807727,37.49751006323854],[127.15550151473776,37.49868492618154],[127.15475982869074,37.499765954521116],[127.15376716570765,37.50071182084856],[127.14881780693477,37.50458482582779],[127.13808756068146,37.519231878965684],[127.12960336025363,37.53460729987978],[127.12884654361109,37.53567944864644],[127.12784148783759,37.536614594132026],[127.1266264772841,37.537377107075194],[127.12524780003933,37.53793793457468],[127.1237579841191,37.538275707688534],[127.12221379516691,37.538377556178474],[127.12067407213678,37.53823959925498],[127.11919748369024,37.53786709355784],[127.11784029110555,37.53727423271366],[127.116654203246,37.536483606134894],[127.11385158823819,37.534209966895006],[127.10299580291483,37.52539923099968],[127.09987228855199,37.52599621419699],[127.09277839989225,37.53883578425843],[127.0920205104564,37.53990818345569],[127.09101420984315,37.54084333733917],[127.08979786837719,37.54160558050512],[127.08841787145545,37.54216584113237],[127.08692685024839,37.542502750417455],[127.08538167344894,37.54260345813283],[127.0838412768446,37.542464123101325],[127.082364413762,37.54209005981312],[127.08100741249643,37.5414955355732],[127.07982202656859,37.54070322594406],[127.07885346005652,37.53974334932646],[127.07813864348138,37.53865251379146],[127.07770482604695,37.53747232026804],[127.07756853784102,37.53624777547993],[127.07759199750569,37.53434330539917],[127.06028052279197,37.520280429752795],[127.05748062738145,37.51800537404833],[127.05653600456212,37.5170756545023],[127.05582972155929,37.516021182290025],[127.04932105238444,37.503578820002986]]

In [47]:
new_geo = []
for g in geo:
    new_geo.append([g[1], g[0]])

In [48]:
folium.Polygon(
    locations = new_geo,
    fill = True,
    tooltip = 'Polygon'
).add_to(map)

In [49]:
map